In [2]:
import os, glob
import numpy as np
import pandas as pd

base_dir     = '/Users/lyanne/EmpathicPain'
run1_pattern = os.path.join(base_dir, 'sub-*_run-01_connectivity.csv')
run1_files   = sorted(glob.glob(run1_pattern))

subject_list = [os.path.basename(f).split('_run-01_connectivity.csv')[0] 
                for f in run1_files]
n_subj = len(subject_list)

conn_all = np.zeros((n_subj, 2, 69, 69))
for i, subj in enumerate(subject_list):
    fn1 = os.path.join(base_dir, f"{subj}_run-01_connectivity.csv")
    fn2 = os.path.join(base_dir, f"{subj}_run-02_connectivity.csv")
    
    # skip the first (header) row
    mat1 = pd.read_csv(fn1, header=None, skiprows=1).values
    mat2 = pd.read_csv(fn2, header=None, skiprows=1).values
    
    assert mat1.shape == (69, 69)
    assert mat2.shape == (69, 69)
    
    conn_all[i, 0] = mat1
    conn_all[i, 1] = mat2

print("conn_all shape:", conn_all.shape)  # → (n_subj, 2, 69, 69)



conn_all shape: (53, 2, 69, 69)


In [8]:
import numpy as np

def threshold_proportional(mat, density):
    """
    Keeps only the top `density` fraction of absolute weights in `mat`.
    mat : square, symmetric 2D array (ROI×ROI)
    density : float between 0 and 1
    returns a new matrix with the same shape, zeroed below threshold
    """
    # 1. copy and zero diagonal
    M = mat.copy()
    np.fill_diagonal(M, 0)
    
    # 2. get all off‐diagonal absolute values (upper triangle)
    iu = np.triu_indices_from(M, k=1)
    abs_vals = np.abs(M[iu])
    
    # 3. find cutoff so that only `density` fraction remain
    cutoff = np.percentile(abs_vals, 100 * (1 - density))
    
    # 4. zero out everything with abs < cutoff
    M[np.abs(M) < cutoff] = 0
    
    # 5. (optional) re‐symmetrize to ensure perfect symmetry
    M = np.maximum(M, M.T)
    return M


# assume conn_all.shape == (n_subj, 2, 69, 69)
n_subj = conn_all.shape[0]
n_runs = conn_all.shape[1]

# pre‐allocate
threshold_conn_all = np.zeros_like(conn_all)

# choose your density
density = 0.20  # e.g. keep top 10% of edges

for i in range(n_subj):
    for run in range(n_runs):
        threshold_conn_all[i, run] = threshold_proportional(
            conn_all[i, run],
            density=density
        )

print("Thresholded connectivity shape:", threshold_conn_all.shape)
# → (n_subj, 2, 69, 69)



Thresholded connectivity shape: (53, 2, 69, 69)
